#### Assignment- Advanced Regression

A US-based housing company named Surprise Housing has decided to enter the Australian market. The company uses data analytics to purchase houses at a price below their actual value and flip them at a higher price. For the same purpose, the company has collected a data set from house sales in Australia. The data is provided in the csv file below.

The company is looking at prospective properties to buy to enter the market.

You are required to build a regression model using regularization, so as to predict the actual value of the prospective properties and decide whether to invest in them or not.

The company wants to know:

1. Which variables are significant in predicting the price of a house

2. How well those variables describe the price of a house




In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler


In [84]:
house_data=pd.read_csv("train.csv")
house_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [85]:
house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [86]:
house_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [87]:
nuni=house_data.nunique()
nuni[nuni==0]

Series([], dtype: int64)

In [88]:
house_data.Alley.value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

In [89]:
house_data.Fence.value_counts()

MnPrv    157
GdPrv     59
GdWo      54
MnWw      11
Name: Fence, dtype: int64

In [90]:
house_data=house_data.drop(columns=['PoolQC','Fence','MiscFeature'])


In [91]:
house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 78 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [92]:
house_data.FireplaceQu.value_counts()

Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: FireplaceQu, dtype: int64

In [93]:
house_data=house_data.drop(columns=['Alley'])

In [94]:
house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [95]:
house_data.FireplaceQu=house_data.FireplaceQu.fillna('NA')


In [96]:
house_data.isnull().sum()[house_data.isnull().sum()>0]

LotFrontage     259
MasVnrType        8
MasVnrArea        8
BsmtQual         37
BsmtCond         37
BsmtExposure     38
BsmtFinType1     37
BsmtFinType2     38
Electrical        1
GarageType       81
GarageYrBlt      81
GarageFinish     81
GarageQual       81
GarageCond       81
dtype: int64

In [97]:
med=house_data.LotFrontage.median()
med

69.0

In [98]:
house_data.LotFrontage=house_data.LotFrontage.fillna(med)

In [99]:
house_data.isnull().sum()[house_data.isnull().sum()>0]

MasVnrType       8
MasVnrArea       8
BsmtQual        37
BsmtCond        37
BsmtExposure    38
BsmtFinType1    37
BsmtFinType2    38
Electrical       1
GarageType      81
GarageYrBlt     81
GarageFinish    81
GarageQual      81
GarageCond      81
dtype: int64

In [100]:
house_data.MasVnrType.value_counts()

None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64

In [101]:
house_data.MasVnrArea.head()

0    196.0
1      0.0
2    162.0
3      0.0
4    350.0
Name: MasVnrArea, dtype: float64

In [102]:
house_data.MasVnrArea=house_data.MasVnrArea.fillna(house_data.MasVnrArea.median())
house_data.MasVnrType=house_data.MasVnrType.fillna('None')


In [103]:
house_data.BsmtQual.value_counts()

TA    649
Gd    618
Ex    121
Fa     35
Name: BsmtQual, dtype: int64

In [104]:
house_data.BsmtQual=house_data.BsmtQual.fillna('NA')

In [105]:
house_data.BsmtCond.value_counts()

TA    1311
Gd      65
Fa      45
Po       2
Name: BsmtCond, dtype: int64

In [106]:
house_data.BsmtCond=house_data.BsmtCond.fillna('NA')

In [107]:
print(house_data.BsmtExposure.value_counts())
print(house_data.BsmtFinType1.value_counts())
print(house_data.BsmtFinType2.value_counts())
print(house_data.Electrical.value_counts())
print(house_data.GarageType.value_counts())
print(house_data.GarageYrBlt.value_counts())
print(house_data.GarageFinish.value_counts())
print(house_data.GarageQual.value_counts())
print(house_data.GarageCond.value_counts())

No    953
Av    221
Gd    134
Mn    114
Name: BsmtExposure, dtype: int64
Unf    430
GLQ    418
ALQ    220
BLQ    148
Rec    133
LwQ     74
Name: BsmtFinType1, dtype: int64
Unf    1256
Rec      54
LwQ      46
BLQ      33
ALQ      19
GLQ      14
Name: BsmtFinType2, dtype: int64
SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64
Attchd     870
Detchd     387
BuiltIn     88
Basment     19
CarPort      9
2Types       6
Name: GarageType, dtype: int64
2005.0    65
2006.0    59
2004.0    53
2003.0    50
2007.0    49
          ..
1927.0     1
1900.0     1
1906.0     1
1908.0     1
1933.0     1
Name: GarageYrBlt, Length: 97, dtype: int64
Unf    605
RFn    422
Fin    352
Name: GarageFinish, dtype: int64
TA    1311
Fa      48
Gd      14
Ex       3
Po       3
Name: GarageQual, dtype: int64
TA    1326
Fa      35
Gd       9
Po       7
Ex       2
Name: GarageCond, dtype: int64


In [108]:
house_data.BsmtExposure = house_data.BsmtExposure.fillna('No')
house_data.BsmtFinType1 = house_data.BsmtFinType1.fillna('NA')
house_data.BsmtFinType2 = house_data.BsmtFinType2.fillna('NA')
house_data.Electrical = house_data.Electrical.fillna(house_data.Electrical.mode()[0])
house_data.GarageType = house_data.GarageType.fillna('NA')
house_data.GarageYrBlt = house_data.GarageYrBlt.fillna(0)
house_data.GarageFinish = house_data.GarageFinish.fillna('NA')
house_data.GarageQual = house_data.GarageQual.fillna('NA')
house_data.GarageCond = house_data.GarageCond.fillna('NA')

In [109]:
house_data.isnull().sum()[house_data.isnull().sum()>0]

Series([], dtype: int64)

In [110]:
house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [111]:
house_data['HouseAge']=house_data.YrSold-house_data.YearBuilt

In [112]:
house_data=house_data.drop(columns=['YrSold','YearBuilt'])


In [113]:
house_data['MSSubClass']

0       60
1       20
2       60
3       70
4       60
        ..
1455    60
1456    20
1457    70
1458    20
1459    20
Name: MSSubClass, Length: 1460, dtype: int64

In [114]:
d={'MSSubClass':object}
house_data=house_data.astype(d)


In [115]:
house_data.ExterQual.value_counts()

TA    906
Gd    488
Ex     52
Fa     14
Name: ExterQual, dtype: int64

In [116]:
house_data=house_data.replace({'ExterQual':{'Po':1,'Fa':2,'Gd':3,'TA':4,'Ex':5}})

In [117]:
house_data.BsmtQual.value_counts()

TA    649
Gd    618
Ex    121
NA     37
Fa     35
Name: BsmtQual, dtype: int64

In [118]:
house_data.BsmtExposure.value_counts()

No    991
Av    221
Gd    134
Mn    114
Name: BsmtExposure, dtype: int64

In [119]:
house_data=house_data.replace({'BsmtQual':{'NA':0,'Po':1,'Fa':2,'Gd':3,'TA':4,'Ex':5}})

In [153]:
house_data=house_data.replace({'BsmtExposure':{'NA':0,'No':1,'Mn':2,'Av':3,'Gd':4},
                              'BsmtFinType1':{'NA':0,'Unf':1,'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6},
                               'HeatingQC':{'Po':1,'Fa':2,'Gd':3,'TA':4,'Ex':5},
                               'KitchenQual':{'Po':1,'Fa':2,'Gd':3,'TA':4,'Ex':5},
                               'FireplaceQu':{'NA':0,'Po':1,'Fa':2,'Gd':3,'TA':4,'Ex':5}
                              })


In [154]:
house_data.BsmtQual.value_counts()

4    649
3    618
5    121
0     37
2     35
Name: BsmtQual, dtype: int64

In [122]:
for c in house_data.columns:
    if house_data[c].dtype==object:
        print(c)
        print(house_data[c].value_counts(normalize=True))

MSSubClass
20     0.367123
60     0.204795
50     0.098630
120    0.059589
30     0.047260
160    0.043151
70     0.041096
80     0.039726
90     0.035616
190    0.020548
85     0.013699
75     0.010959
45     0.008219
180    0.006849
40     0.002740
Name: MSSubClass, dtype: float64
MSZoning
RL         0.788356
RM         0.149315
FV         0.044521
RH         0.010959
C (all)    0.006849
Name: MSZoning, dtype: float64
Street
Pave    0.99589
Grvl    0.00411
Name: Street, dtype: float64
LotShape
Reg    0.633562
IR1    0.331507
IR2    0.028082
IR3    0.006849
Name: LotShape, dtype: float64
LandContour
Lvl    0.897945
Bnk    0.043151
HLS    0.034247
Low    0.024658
Name: LandContour, dtype: float64
Utilities
AllPub    0.999315
NoSeWa    0.000685
Name: Utilities, dtype: float64
LotConfig
Inside     0.720548
Corner     0.180137
CulDSac    0.064384
FR2        0.032192
FR3        0.002740
Name: LotConfig, dtype: float64
LandSlope
Gtl    0.946575
Mod    0.044521
Sev    0.008904
Name: LandSlop

In [123]:
house_data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'Open

In [155]:
# drop columns which have more than 80% bias data
house_data=house_data.drop(columns=['Id','Street', 'LandContour', 'Utilities', 'LandSlope', 'Condition2', 'RoofMatl', 'BsmtCond','Heating', 'CentralAir', 'Electrical', 'Functional', 'GarageQual', 'GarageCond', 'PavedDrive', 'Condition1', 'BldgType', 'ExterCond', 'BsmtFinType2', 'SaleType', 'SaleCondition'])
house_data.info()

KeyError: "['Id' 'Street' 'LandContour' 'Utilities' 'LandSlope' 'Condition2'\n 'RoofMatl' 'BsmtCond' 'Heating' 'CentralAir' 'Electrical' 'Functional'\n 'GarageQual' 'GarageCond' 'PavedDrive' 'Condition1' 'BldgType'\n 'ExterCond' 'BsmtFinType2' 'SaleType' 'SaleCondition'] not found in axis"

#### Create Dummy Variables

In [125]:
MSSubClass_df=pd.get_dummies(house_data['MSSubClass'], drop_first=True, prefix='MSSubClass')
MSSubClass_df

C:\Users\Shree\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1456,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1457,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1458,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [126]:
house_data=pd.concat([house_data,MSSubClass_df],axis=1)
house_data

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LotConfig,Neighborhood,HouseStyle,OverallQual,OverallCond,...,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190
0,60,RL,65.0,8450,Reg,Inside,CollgCr,2Story,7,5,...,1,0,0,0,0,0,0,0,0,0
1,20,RL,80.0,9600,Reg,FR2,Veenker,1Story,6,8,...,0,0,0,0,0,0,0,0,0,0
2,60,RL,68.0,11250,IR1,Inside,CollgCr,2Story,7,5,...,1,0,0,0,0,0,0,0,0,0
3,70,RL,60.0,9550,IR1,Corner,Crawfor,2Story,7,5,...,0,1,0,0,0,0,0,0,0,0
4,60,RL,84.0,14260,IR1,FR2,NoRidge,2Story,8,5,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Reg,Inside,Gilbert,2Story,6,5,...,1,0,0,0,0,0,0,0,0,0
1456,20,RL,85.0,13175,Reg,Inside,NWAmes,1Story,6,6,...,0,0,0,0,0,0,0,0,0,0
1457,70,RL,66.0,9042,Reg,Inside,Crawfor,2Story,7,9,...,0,1,0,0,0,0,0,0,0,0
1458,20,RL,68.0,9717,Reg,Inside,NAmes,1Story,5,6,...,0,0,0,0,0,0,0,0,0,0


In [127]:
MSZoning_df=pd.get_dummies(house_data['MSZoning'], drop_first=True, prefix='MSZoning')
MSZoning_df

,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0
...,...,...,...,...
1455,0,0,1,0
1456,0,0,1,0
1457,0,0,1,0
1458,0,0,1,0


In [128]:
house_data=pd.concat([house_data,MSZoning_df],axis=1)

In [129]:
LotShape_df=pd.get_dummies(house_data['LotShape'], drop_first=True, prefix='LotShape')
house_data=pd.concat([house_data,LotShape_df],axis=1)

In [130]:
# get all object type columns
object_columns=[]
for c in house_data.columns:
    if house_data[c].dtype==object:
        object_columns.append(c)
object_columns        

['MSSubClass',
 'MSZoning',
 'LotShape',
 'LotConfig',
 'Neighborhood',
 'HouseStyle',
 'RoofStyle',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'Foundation',
 'FireplaceQu',
 'GarageType',
 'GarageFinish']

In [131]:
LotShape_df=pd.get_dummies(house_data['LotShape'], drop_first=True, prefix='LotShape')
house_data=pd.concat([house_data,LotShape_df],axis=1)

In [132]:
LotConfig_df=pd.get_dummies(house_data['LotConfig'], drop_first=True, prefix='LotConfig')
house_data=pd.concat([house_data,LotConfig_df],axis=1)

In [133]:
Neighborhood_df=pd.get_dummies(house_data['Neighborhood'], drop_first=True, prefix='Neighborhood')
house_data=pd.concat([house_data,Neighborhood_df],axis=1)

In [134]:
HouseStyle_df=pd.get_dummies(house_data['HouseStyle'], drop_first=True, prefix='HouseStyle')
house_data=pd.concat([house_data,HouseStyle_df],axis=1)

In [135]:
RoofStyle_df=pd.get_dummies(house_data['RoofStyle'], drop_first=True, prefix='RoofStyle')
house_data=pd.concat([house_data,RoofStyle_df],axis=1)

In [136]:
Exterior1st_df=pd.get_dummies(house_data['Exterior1st'], drop_first=True, prefix='Exterior1st')
house_data=pd.concat([house_data,Exterior1st_df],axis=1)

In [137]:
Exterior2nd_df=pd.get_dummies(house_data['Exterior2nd'], drop_first=True, prefix='Exterior2nd')
house_data=pd.concat([house_data,Exterior2nd_df],axis=1)

In [138]:
MasVnrType_df=pd.get_dummies(house_data['MasVnrType'], drop_first=True, prefix='MasVnrType')
house_data=pd.concat([house_data,MasVnrType_df],axis=1)

In [139]:
Foundation_df=pd.get_dummies(house_data['Foundation'], drop_first=True, prefix='Foundation')
house_data=pd.concat([house_data,Foundation_df],axis=1)

In [140]:
FireplaceQu_df=pd.get_dummies(house_data['FireplaceQu'], drop_first=True, prefix='FireplaceQu')
house_data=pd.concat([house_data,FireplaceQu_df],axis=1)

In [141]:
GarageType_df=pd.get_dummies(house_data['GarageType'], drop_first=True, prefix='GarageType')
house_data=pd.concat([house_data,GarageType_df],axis=1)

In [142]:
GarageFinish_df=pd.get_dummies(house_data['GarageFinish'], drop_first=True, prefix='GarageFinish')
house_data=pd.concat([house_data,GarageFinish_df],axis=1)

In [143]:
house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 170 entries, MSSubClass to GarageFinish_Unf
dtypes: float64(3), int64(38), object(14), uint8(115)
memory usage: 791.4+ KB


In [148]:
house_data=house_data.drop(columns=['MSSubClass', 'MSZoning', 'LotShape', 'LotConfig', 'Neighborhood', 'HouseStyle'
                                    , 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
                                    'GarageType', 'GarageFinish'])

In [149]:
house_data.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,ExterQual,BsmtQual,BsmtExposure,BsmtFinType1,...,FireplaceQu_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,GarageFinish_NA,GarageFinish_RFn,GarageFinish_Unf
0,65.0,8450,7,5,2003,196.0,3,3,1,6,...,0,1,0,0,0,0,0,0,1,0
1,80.0,9600,6,8,1976,0.0,4,3,4,5,...,1,1,0,0,0,0,0,0,1,0
2,68.0,11250,7,5,2002,162.0,3,3,2,6,...,1,1,0,0,0,0,0,0,1,0
3,60.0,9550,7,5,1970,0.0,4,4,1,5,...,0,0,0,0,0,1,0,0,0,1
4,84.0,14260,8,5,2000,350.0,3,3,3,6,...,1,1,0,0,0,0,0,0,1,0


In [157]:
df_train,df_test = train_test_split(house_data,train_size=0.70,random_state=100)
print(df_train.shape)
print(df_test.shape)

(1021, 157)
(439, 157)


#### Recalling the numeric factors

In [158]:
scaler=StandardScaler()

#create list of numeric variables
num_vars = ['LotFrontage','LotArea','OverallQual','OverallCond','YearRemodAdd','MasVnrArea', 'ExterQual', 'BsmtQual'
            , 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'TotalBsmtSF'
           , 'HeatingQC', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath'
            , 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces'
           , 'FireplaceQu', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch'
           , '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'HouseAge']

df_train[num_vars]=scaler.fit_transform(df_train[num_vars])
df_train.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,ExterQual,BsmtQual,BsmtExposure,BsmtFinType1,...,FireplaceQu_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,GarageFinish_NA,GarageFinish_RFn,GarageFinish_Unf
318,0.925326,-0.057261,0.648310,-0.532073,0.389207,0.809137,-1.240543,-0.559287,2.181065,1.138412,...,1,1,0,0,0,0,0,0,1,0
239,-0.795709,-0.169753,-0.066511,-1.448170,-1.698692,-0.558025,0.576666,0.551671,-0.653302,-0.753845,...,0,0,0,0,0,1,0,0,0,1
986,-0.478677,-0.502762,-0.066511,2.216221,0.874765,-0.558025,0.576666,0.551671,-0.653302,-1.226909,...,0,1,0,0,0,0,0,0,0,1
1416,-0.433386,0.082504,-1.496155,0.384025,-1.698692,-0.558025,0.576666,0.551671,-0.653302,-1.226909,...,0,0,0,0,0,1,0,0,0,1
390,-0.886290,-0.202365,-0.781333,2.216221,-1.698692,-0.558025,0.576666,0.551671,-0.653302,-0.280781,...,0,0,0,0,0,1,0,0,0,1


In [159]:
df_train.describe()

,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,ExterQual,BsmtQual,BsmtExposure,BsmtFinType1,...,FireplaceQu_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,GarageFinish_NA,GarageFinish_RFn,GarageFinish_Unf
count,1.021000e+03,1.021000e+03,1.021000e+03,1.021000e+03,1.021000e+03,1.021000e+03,1.021000e+03,1.021000e+03,1.021000e+03,1.021000e+03,...,1021.000000,1021.000000,1021.000000,1021.000000,1021.000000,1021.000000,1021.000000,1021.000000,1021.000000,1021.000000
mean,-1.043892e-17,-7.655211e-17,-2.696722e-16,1.539741e-16,-8.420732e-16,-6.611318e-17,-1.618033e-16,-1.565839e-16,6.263354e-17,8.612112e-17,...,0.203722,0.589618,0.011753,0.063663,0.006856,0.269344,0.055828,0.055828,0.278159,0.421156
std,1.000490e+00,1.000490e+00,1.000490e+00,1.000490e+00,1.000490e+00,1.000490e+00,1.000490e+00,1.000490e+00,1.000490e+00,1.000490e+00,...,0.402962,0.492144,0.107826,0.244271,0.082557,0.443836,0.229701,0.229701,0.448312,0.493986
min,-2.199712e+00,-8.747892e-01,-3.640619e+00,-4.196464e+00,-1.698692e+00,-5.580245e-01,-3.057753e+00,-3.892161e+00,-6.533017e-01,-1.699973e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-4.333861e-01,-2.951529e-01,-7.813329e-01,-5.320725e-01,-8.732432e-01,-5.580245e-01,-1.240543e+00,-5.592874e-01,-6.533017e-01,-1.226909e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-2.577250e-02,-9.559968e-02,-6.651132e-02,-5.320725e-01,4.377630e-01,-5.580245e-01,5.766658e-01,5.516706e-01,-6.533017e-01,1.922837e-01,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.818411e-01,9.919770e-02,6.483103e-01,3.840254e-01,9.233208e-01,2.857702e-01,5.766658e-01,5.516706e-01,1.236276e+00,1.138412e+00,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,1.102509e+01,1.987332e+01,2.792775e+00,3.132319e+00,1.214656e+00,7.986733e+00,2.393875e+00,1.662629e+00,2.181065e+00,1.138412e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### Training Model

In [160]:
y_train=df_train.pop('SalePrice')
X_train=df_train

In [161]:
X_train.columns

Index(['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearRemodAdd',
       'MasVnrArea', 'ExterQual', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1',
       ...
       'FireplaceQu_TA', 'GarageType_Attchd', 'GarageType_Basment',
       'GarageType_BuiltIn', 'GarageType_CarPort', 'GarageType_Detchd',
       'GarageType_NA', 'GarageFinish_NA', 'GarageFinish_RFn',
       'GarageFinish_Unf'],
      dtype='object', length=156)

In [162]:
#recursive feature elimination
from sklearn.feature_selection import RFE

In [164]:
lm=LinearRegression()
lm.fit(X_train, y_train)


LinearRegression()

In [167]:
rfe=RFE(lm)
rfe=rfe.fit(X_train, y_train)

In [168]:
cols=X_train.columns[rfe.support_]
cols

Index(['LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 'BsmtQual',
       'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'KitchenAbvGr', 'FireplaceQu', 'GarageCars', 'HouseAge',
       'MSSubClass_45', 'MSSubClass_160', 'LotShape_IR2', 'LotShape_IR3',
       'LotShape_Reg', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg',
       'LotConfig_CulDSac', 'LotConfig_FR2', 'Neighborhood_CollgCr',
       'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_Mitchel',
       'Neighborhood_NAmes', 'Neighborhood_NWAmes', 'Neighborhood_NridgHt',
       'Neighborhood_OldTown', 'Neighborhood_SWISU', 'Neighborhood_SawyerW',
       'Neighborhood_Timber', 'HouseStyle_1.5Unf', 'HouseStyle_1Story',
       'HouseStyle_2.5Fin', 'HouseStyle_2.5Unf', 'RoofStyle_Gambrel',
       'Exterior1st_AsphShn', 'Exterior1st_CBlock', 'Exterior1st_CemntBd',
       'Exterior1st_HdBoard', 'Exterior1st_I

In [169]:
#Skipped columns from rfe
X_train.columns[~rfe.support_]

Index(['LotFrontage', 'YearRemodAdd', 'ExterQual', 'BsmtFinType1', 'HeatingQC',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'MSSubClass_30', 'MSSubClass_40', 'MSSubClass_50',
       'MSSubClass_60', 'MSSubClass_70', 'MSSubClass_75', 'MSSubClass_80',
       'MSSubClass_85', 'MSSubClass_90', 'MSSubClass_120', 'MSSubClass_180',
       'MSSubClass_190', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL',
       'MSZoning_RM', 'LotConfig_FR3', 'LotConfig_Inside',
       'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide',
       'Neighborhood_ClearCr', 'Neighborhood_Crawfor', 'Neighborhood_IDOTRR',
       'Neighborhood_MeadowV', 'Neighborhood_NPkVill', 'Neighborhood_NoRidge',
       'Neighborhood_Sawyer', 'Neighborhood_Somerst', 'Neighborhood_StoneBr',
       'Ne

In [171]:
X_train_rfe=X_train[cols]

In [173]:
import statsmodels.api as sm

In [174]:
X_train_rfe=sm.add_constant(X_train_rfe)


In [175]:
lm=sm.OLS(y_train, X_train_rfe).fit()


In [176]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     80.58
Date:                Fri, 31 Mar 2023   Prob (F-statistic):               0.00
Time:                        10:14:21   Log-Likelihood:                -11988.
No. Observations:                1021   AIC:                         2.411e+04
Df Residuals:                     952   BIC:                         2.445e+04
Df Model:                          68                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 1.307e+05   1.01e+04     12.893      0.000    1.11e+05    1.51e+05
LotArea               5285.8819   1218.165      4.339      0.000    2895.283    7676.481
OverallQual           1.966e+04   1877.072     10.476      0.000     1.6e+04    2.33e+04
OverallCond           6511.1207   1181.622      5.510      0.000    4192.236    8830.006
MasVnrArea            5925.3720   1501.734      3.946      0.000    2978.280    8872.464
BsmtQual              5403.1096   1289.962      4.189      0.000    2871.613    7934.606
BsmtExposure          7626.3878   1274.944      5.982      0.000    5124.362    1.01e+04
BsmtFinSF1             380.7061   1180.088      0.323      0.747   -1935.168    2696.580
BsmtFinSF2           -1174.6379   1046.251     -1.123      0.262   -3227.863     878.588
BsmtUnfSF            -1644.2896   1037.696     -1.585      0.113   -3680.725     392.146
TotalBsmtSF          -1658.6195   1574.871     -1.053      0.293   -4749.239    1432.000
1stFlrSF              8917.2042   1739.595      5.126      0.000    5503.321    1.23e+04
2ndFlrSF              1.556e+04   1508.944     10.311      0.000    1.26e+04    1.85e+04
LowQualFinSF          -624.0843   1183.613     -0.527      0.598   -2946.877    1698.708
GrLivArea             1.935e+04   1117.002     17.328      0.000    1.72e+04    2.15e+04
BsmtFullBath          5230.9632   1402.345      3.730      0.000    2478.919    7983.007
KitchenAbvGr         -5148.2555   1218.011     -4.227      0.000   -7538.552   -2757.959
FireplaceQu           2.996e+04   3905.189      7.672      0.000    2.23e+04    3.76e+04
GarageCars            1.197e+04   1731.624      6.911      0.000    8569.761    1.54e+04
HouseAge             -1.248e+04   2270.458     -5.495      0.000   -1.69e+04   -8021.530
MSSubClass_45        -1.403e+04   2.47e+04     -0.569      0.570   -6.24e+04    3.44e+04
MSSubClass_160       -2.703e+04   5868.719     -4.606      0.000   -3.86e+04   -1.55e+04
LotShape_IR2          -142.5091   1573.289     -0.091      0.928   -3230.024    2945.006
LotShape_IR2          -142.5091   1573.289     -0.091      0.928   -3230.024    2945.006
LotShape_IR3         -1.082e+04   2938.225     -3.681      0.000   -1.66e+04   -5049.674
LotShape_IR3         -1.082e+04   2938.225     -3.681      0.000   -1.66e+04   -5049.674
LotShape_Reg           -47.4062    624.492     -0.076      0.940   -1272.947    1178.135
LotShape_Reg           -47.4062    624.492     -0.076      0.940   -1272.947    1178.135
LotShape_IR2          -142.5091   1573.289     -0.091      0.928   -3230.024    2945.006
LotShape_IR2          -142.5091   1573.289     -0.091      0.928   -3230.024    2945.006
LotShape_IR3         -1.082e+04   2938.225     -3.681      0.000   -1.66e+04   -5049.674
LotShape_IR3         -1.082e+04   2938.225     -3.681    